<div dir='rtl'>
در این قسمت به سراغ استفاده از روش بردار پشتیبان می‌رویم.
<br>
با توجه یه اینکه ویژگی‌های داده بیشتر از نوع دسته‌بندی هستند، انتظار زیادی از این روش روی این داده نمی‌رود.
</div>

In [22]:
import pandas as ps
import numpy as np
import seaborn as sns
from sklearn import svm

import sys
IN_COLAB = 'google.colab' in sys.modules


In [24]:

data = ps.read_csv('/content/gdrive/My Drive/train_data.csv' if IN_COLAB else 'train_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768416 entries, 0 to 3768415
Data columns (total 16 columns):
 #   Column        Dtype
---  ------        -----
 0   displayId     int64
 1   timestamp     int64
 2   dayOfWeek     int64
 3   hourOfDay     int64
 4   advertiserId  int64
 5   campaignId    int64
 6   creativeId    int64
 7   publisher     int64
 8   widgetId      int64
 9   device        int64
 10  os            int64
 11  browser       int64
 12  source        int64
 13  docId         int64
 14  userId        int64
 15  clicked       int64
dtypes: int64(16)
memory usage: 460.0 MB


<div dir='rtl'>ابتدا به سراغ تقسیم‌بندی داده‌ها می‌رویم. با توجه به نامتوازن بودن داده‌ها، روش‌هایی مانند (undersampling) نمونه‌زایی (oversampling) و نمونه‌کاهی می‌تواند کمک کند اما به توجه به هزینه این روش‌ها که دور شدن از داده‌واقعی است، تلاش خود را بر اصلاح تابع خطا به شکلی که داده‌های کلاس نادر را مهمتر درنظر گیرد می‌گذاریم.
<br>
کتابخانه sklearn الگوریتم svm را به صورت وزن دار پیاده‌سازی کرده است. به طوری‌که میتوان با وزن‌دادن به کلاس‌ها، تشخیص درست‌ یک کلاس را بر دیگر کلاس‌ها اولویت داد. با توجه به این‌که تعداد نمایش‌های منجر به مراجعه، ۲۵ درصد کل داده است، وزن این کلاس راتعیین می‌کنیم.
<div>

In [20]:
clsf = svm.SVC(kernel='linear', C=1, class_weight={0:.25, 1:.75})

<div dir='rtl'>
اکنون عملیات روی داده‌ها را شروع می‌کنیم.
<br>
اول ویژگی‌های غیر ترتیبی را حذف می‌کنیم و بعد داده‌ی یادگیری را از داده‌ آزمون جدا می‌کنیم.
</div>

In [16]:
from sklearn.model_selection import train_test_split

clean_data = data.drop(data.columns[[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]] , axis=1)
print(clean_data.info())
train, test = train_test_split(clean_data, test_size=0.2)
train, test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768416 entries, 0 to 3768415
Data columns (total 3 columns):
 #   Column     Dtype
---  ------     -----
 0   dayOfWeek  int64
 1   hourOfDay  int64
 2   clicked    int64
dtypes: int64(3)
memory usage: 86.3 MB
None


(         dayOfWeek  hourOfDay  clicked
 3377798          2         13        0
 2813550          0          9        0
 11720            4          3        0
 48759            4          5        0
 218295           4         13        0
 ...            ...        ...      ...
 98499            4          7        0
 443508           5          7        0
 2964473          0         21        0
 1140963          0         21        0
 1206560          1          5        0
 
 [3014732 rows x 3 columns],
          dayOfWeek  hourOfDay  clicked
 3662148          3         14        0
 2162988          4          6        1
 1809093          2         22        0
 1772098          2         18        0
 68838            4          6        0
 ...            ...        ...      ...
 1535623          2          5        0
 812810           6         16        0
 1779748          2         19        0
 2329906          4         19        0
 2434953          5          8        0
 
 [75368

<div dir=rtl'>
حالا به سراغ خود الگوریتم می‌رویم.
<br>

</div>

In [19]:

clsf.fit(train[['dayOfWeek','hourOfDay']], train[['clicked']])
clsf.predict(test[['dayOfWeek','hourOfDay']])
# (clsf.predict(newdata[0:2]), data[1:2].clicked)

,clicked
3377798,0
2813550,0
11720,0
48759,0
218295,0
...,...
98499,0
443508,0
2964473,0
1140963,0
